In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
num_robots = 4
sampling_rate = 5
dataset = 'bathymetry'

filename = f'../datasets/results/{dataset}_{num_robots}R_{sampling_rate}S.json'
save_filename = filename[:-5] + '-{}.pdf'
results = json.load(open(filename, 'r'))

rmse = defaultdict(list)
rmse_std = defaultdict(list)

param_time = defaultdict(list)
param_time_std = defaultdict(list)

ipp_time = defaultdict(list)
ipp_time_std = defaultdict(list)

for num_sensors in results.keys():
    for method in  results[num_sensors].keys():
        #if 'Agg' in method:
        #    continue
        for metric in results[num_sensors][method].keys():
            if metric=='RMSE':
                rmse[method].append(np.mean(results[num_sensors][method][metric]))
                rmse_std[method].append(np.std(results[num_sensors][method][metric]))
            elif metric=='ParamTime':
                param_time[method].append(np.mean(results[num_sensors][method][metric]))
                param_time_std[method].append(np.std(results[num_sensors][method][metric]))
            elif metric=='IPPTime':
                ipp_time[method].append(np.mean(results[num_sensors][method][metric]))
                ipp_time_std[method].append(np.std(results[num_sensors][method][metric]))

xrange = list(results.keys())

colors = ['C2', 'C3', 'C0', 'C1', 'C4', 'C5']
datasets = ['intel', 'precipitation', 'soil', 'salinity']

plt.figure()
for i, key in enumerate(rmse.keys()):
    label = 'Adaptive-SGP (Ours)' if key=='Adaptive-SGP' else key
    plt.plot(xrange, rmse[key], label=label, c=colors[i])
    plt.fill_between(xrange,
                     np.array(rmse[key])+rmse_std[key], 
                     np.array(rmse[key])-rmse_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend()
plt.xlabel("Number of Waypoints")
plt.ylabel("RMSE")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S-RMSE.pdf', bbox_inches='tight')
plt.show()

plt.figure()
for i, key in enumerate(ipp_time.keys()):
    label = 'Adaptive-SGP (Ours)' if key=='Adaptive-SGP' else key
    plt.plot(xrange, ipp_time[key], label=label, c=colors[i])
    plt.fill_between(xrange,
                     np.array(ipp_time[key])+ipp_time_std[key], 
                     np.array(ipp_time[key])-ipp_time_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend()
plt.xlabel("Number of Waypoints")
plt.ylabel("IPP Runtime (s)")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S-IPP.pdf', bbox_inches='tight')
plt.show()

plt.figure()
for i, key in enumerate(param_time.keys()):
    label = 'Adaptive-SGP (Ours)' if key=='Adaptive-SGP' else key
    plt.plot(xrange, param_time[key], label=label, c=colors[i])
    plt.fill_between(xrange,
                     np.array(param_time[key])+param_time_std[key], 
                     np.array(param_time[key])-param_time_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend()
plt.xlabel("Number of Waypoints")
plt.ylabel("Parameter Learning Runtime (s)")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S-Param.pdf', bbox_inches='tight')
plt.show()